<div class="alert alert-block alert-info">
<b>Note:</b> To run this notebook, MMTSB needs to be installed
    
</div>

In [1]:
import os
import sys
import subprocess
import numpy as np
import pandas as pd
import shlex

import pycharmm
import pycharmm.read as read
import pycharmm.lingo as lingo
import pycharmm.generate as gen
import pycharmm.settings as settings
import pycharmm.write as write
import pycharmm.nbonds as nbonds
import pycharmm.ic as ic
import pycharmm.coor as coor
import pycharmm.energy as energy
import pycharmm.dynamics as dyn
import pycharmm.minimize as minimize
import pycharmm.crystal as crystal
import pycharmm.select as select
import pycharmm.image as image
import pycharmm.psf as psf
import pycharmm.param as param
import pycharmm.cons_harm as cons_harm
import pycharmm.cons_fix as cons_fix
import pycharmm.shake as shake
import pycharmm.scalar as scalar
import pycharmm.charmm_file as charmm_file

The following variables need to be defined

In [2]:
fname_base='kaaeh'    # system name
seq_str='LYS ALA ALA GLU HSP'
out_dir='./test2'


In [3]:
prm_dir='./toppar_c36_jul21'
out_fn=f'{fname_base}-3state'
os.system(f'mkdir -p {out_dir}')
log_fn=out_dir+'/charmm.out'

In [4]:
log_unit=20
segid='PROT'   # segment ID for the peptide

In [5]:
def open_clog():
    '''
    specify charmm output file for a given system
    '''
    clog=charmm_file.CharmmFile(file_name=log_fn,file_unit=log_unit,read_only=False,formatted=True)
    lingo.charmm_script('outu {}'.format(log_unit))

In [6]:
def read_param():
    read.rtf(prm_dir+'/top_all36_prot.rtf')
    read.rtf(prm_dir+'/cphmd_patch.rtf',append=True)
    read.prm(prm_dir+'/par_all36m_prot.prm',flex=True)
    lingo.charmm_script('stream '+prm_dir+'/toppar_water_ions.str')

In [7]:
def gen_init():
    read.sequence_string(seq_str)
    gen.new_segment(seg_name=segid,first_patch='ACE',last_patch='CT3',setup_ic=True)
    ic.prm_fill(replace_all=True)
    ic.seed(1, 'CAY', 1, 'CY', 1, 'N')
    ic.build()
    lingo.charmm_script('Hbuild')
    write.psf_card(f'{out_dir}/{fname_base}.psf')
    write.coor_pdb(f'{out_dir}/{fname_base}.pdb')
    num_chrg=round(lingo.get_energy_value('CGTOT'))
    return num_chrg



In [8]:
def switch_atom_names(resn):
    if resn == 'ASP':
       lingo.charmm_script('''
            rename atome OD1T sele type OD1 .and. resn ASP end
            rename atome OD2T sele type OD2 .and. resn ASP end
            rename atome HD2T sele type HD2 .and. resn ASP end
            rename atome OD2 sele type OD1T .and. resn ASP end
            rename atome OD1 sele type OD2T .and. resn ASP end
            rename atome HD1 sele type HD2T .and. resn ASP end
       ''')
    elif resn == 'GLU':
       lingo.charmm_script('''
            rename atome OE1T sele type OE1 .and. resn GLU end
            rename atome OE2T sele type OE2 .and. resn GLU end
            rename atome HE2T sele type HE2 .and. resn GLU end
            rename atome OE2 sele type OE1T .and. resn GLU end
            rename atome OE1 sele type OE2T .and. resn GLU end
            rename atome HE1 sele type HE2T .and. resn GLU end
       ''')


In [9]:
def remove_suffix(resn,suffix):
    '''
    remove suffix from titratable groups
    '''
    atom_list=titr_grp(resn.upper())
    for name in atom_list:
        lingo.charmm_script('rename atom {} sele type {}{} .and. resn {} end'.format(name,name,suffix,resn))



In [10]:
class state1_o2:
    '''
    generate state1 structure: protonation & add atom name suffix.
    add H at OD2 position for ASP, or OE2 position for GLU.
    by default, this is what charmm36m does when patching. 
    '''
    def __init__(self,titr_res):
        self.titr_res_dict=titr_res
        self.asp_patch()
        self.glu_patch()
        self.reset_ic()
        self.asp_edit_ic()
        self.glu_edit_ic()
        lingo.charmm_script('ic build')
        write.psf_card(out_dir+'/'+fname_base+'-state1-o2.psf')
        add_suffix('ASP', 'p')
        add_suffix('GLU', 'm')
        add_suffix('HSP', 'u')
        add_suffix('LYS', 'm')
        write.coor_pdb(out_dir+'/'+fname_base+'-state1-o2.pdb')
    def asp_patch(self):
        resid_list=self.titr_res_dict['ASP'][0]
        for i in range(0,len(resid_list)):
            resid=resid_list[i]
            self.patch('ASP',resid)
    def glu_patch(self):
        resid_list=self.titr_res_dict['GLU'][0]
        for i in range(0,len(resid_list)):
            resid=resid_list[i]
            self.patch('GLU',resid)
    def patch(self,resn,resid):
        lingo.charmm_script('''
              patch {}p {} {}
        '''.format(resn,segid,resid))
    def reset_ic(self):
        lingo.charmm_script('''
              autogen angles dihed
              hbuild
              ic gene
              ic fill
        ''')
    def edit_ic(self,resn,resid,resinx):
        if resn == 'ASP':
           ic.edit_dihedral(resinx,'CB',resinx,'CG',resinx,'OD2',resinx,'HD2',180)
           lingo.charmm_script('coor init sele type HD2 .and. resid {} end'.format(resid))
        elif resn == 'GLU':
           ic.edit_dihedral(resinx,'CG',resinx,'CD',resinx,'OE2',resinx,'HE2',180)
           lingo.charmm_script('coor init sele type HE2 .and. resid {} end'.format(resid))
    def asp_edit_ic(self):
        resid_list=self.titr_res_dict['ASP'][0]
        resinx_list=self.titr_res_dict['ASP'][1]
        for i in range(0,len(resinx_list)):
            resid=resid_list[i]
            resinx=int(resinx_list[i])+1
            self.edit_ic('ASP',resid,resinx)
    def glu_edit_ic(self):
        resid_list=self.titr_res_dict['GLU'][0]
        resinx_list=self.titr_res_dict['GLU'][1]
        for i in range(0,len(resinx_list)):
            resid=resid_list[i]
            resinx=int(resinx_list[i])+1
            self.edit_ic('GLU',resid,resinx)



In [11]:
class patch_all:
    '''
    generate psf and pdb for the hybrid model (protein only)
    '''
    def __init__(self,titr_res,ns):
        self.titr_res_dict=titr_res
        self.patch(ns)
        self.del_connect()
        write.psf_card(f'{out_dir}/{out_fn}.psf')
        settings.set_bomb_level(-1)
        read.pdb(out_dir+'/'+fname_base+'-state1-o2.pdb',resid=True)
        read.pdb(out_dir+'/'+fname_base+'-state2-o1.pdb',resid=True)
        settings.set_bomb_level(0)
        coor_shift(self.titr_res_dict)
        write.coor_pdb(f'{out_dir}/{out_fn}.pdb')
    def patch(self,ns):
        ns=int(ns)
        # to bypass the error of "MISSING PARAMETERS"
        settings.set_bomb_level(-2)
        for resn in list(self.titr_res_dict.keys()):
            AA=resn.upper()
            resid_list=self.titr_res_dict[resn][0]
            for resid in resid_list:
                if ns==2:
                   if AA=='ASP':
                      lingo.charmm_script('''
                            patch ASPP1 %s %s setup
                      '''%(segid,resid))
                   elif AA=='GLU':
                      lingo.charmm_script('''
                            patch GLUP1 %s %s setup
                      '''%(segid,resid))
                elif ns==3:
                   if AA=='ASP':
                      lingo.charmm_script('''
                            patch ASPP1 %s %s setup
                            patch ASPP2 %s %s setup
                      '''%(segid,resid,segid,resid))
                   elif AA=='GLU':
                      lingo.charmm_script('''
                            patch GLUP1 %s %s setup
                            patch GLUP2 %s %s setup
                      '''%(segid,resid,segid,resid))
                if AA=='HSP':
                   lingo.charmm_script('''
                         patch HSDN %s %s setup
                         patch HSEN %s %s setup
                   '''%(segid,resid,segid,resid))
                elif AA=='LYS':
                   lingo.charmm_script('''
                         patch LYSN %s %s setup
                   '''%(segid,resid))
        lingo.charmm_script('autogen angles dihed')
        settings.set_bomb_level(0)
    def del_connect(self):    
        for resn in list(self.titr_res_dict.keys()):
            AA=resn.upper()
            resid_list=self.titr_res_dict[resn][0]
            for resid in resid_list:
                atom_list=titr_grp(AA)
                sele1=~pycharmm.SelectAtoms(select_all=True)
                sele2=~pycharmm.SelectAtoms(select_all=True)
                sele3=~pycharmm.SelectAtoms(select_all=True)
                if AA=='HSP':
                   suffix1='u'
                   suffix2='m'
                else:
                   suffix1='m'
                   suffix2='p'
                for name in atom_list:
                    sele1 = sele1 | pycharmm.SelectAtoms(atom_type=name+'W')
                    sele2 = sele2 | pycharmm.SelectAtoms(atom_type=name+suffix1.upper())
                    sele3 = sele3 | pycharmm.SelectAtoms(atom_type=name+suffix2.upper())
                psf.delete_connectivity(sele1,sele2) 
                psf.delete_connectivity(sele1,sele3) 
                psf.delete_connectivity(sele2,sele3) 


In [12]:
class state2_o1:
    '''
    generate state2 structure: very similar to state1 but w/ different atom name suffix.
    add H at OD1 position for ASP, or OE1 position for GLU.
    '''
    def __init__(self):
        self.protonation_o1('ASP','p')
        self.protonation_o1('GLU','m')
        self.protonation_o1('HSP','u')
        self.protonation_o1('LYS','m')
        write.coor_pdb(out_dir+'/tmp.pdb')
        lingo.charmm_script('delete atom sele all end')
        read.psf_card(out_dir+'/'+fname_base+'-state1-o2.psf')
        read.pdb(out_dir+'/tmp.pdb',resid=True)
        lingo.charmm_script('hbuild')
        switch_atom_names('ASP')
        switch_atom_names('GLU')
        add_suffix('ASP','m')
        add_suffix('GLU','p')
        add_suffix('HSP','m')
        add_suffix('LYS','p')
        write.coor_pdb(out_dir+'/'+fname_base+'-state2-o1.pdb')
        os.system('rm -f '+out_dir+'/tmp.pdb')
    def protonation_o1(self,selresn,suffix_old):
        selresn=selresn.upper()
        atom_list=titr_grp(selresn)
        # remove old suffix
        remove_suffix(selresn,suffix_old)
        if selresn == 'ASP' or selresn == 'GLU':
           # delete the newly added H
           lingo.charmm_script('''
                 bomblev -1
                 delete atom sele resn {} .and. ( type HD2 .or. type HE2 ) end
                 bomblev 0 
           '''.format(selresn))
           # switch atom names
           switch_atom_names(selresn)


In [13]:
def find_titr_res():
    '''
    Find titratable residues in a PDB structure.
    Return a dictionary of residue IDs and names.
    '''
    sele_ca=pycharmm.SelectAtoms(atom_type='CA')
    sele_asp=pycharmm.SelectAtoms(res_name='ASP')
    sele_glu=pycharmm.SelectAtoms(res_name='GLU')
    sele_hsp=pycharmm.SelectAtoms(res_name='HSP')
    sele_hsd=pycharmm.SelectAtoms(res_name='HSD')
    sele_hse=pycharmm.SelectAtoms(res_name='HSE')
    sele_lys=pycharmm.SelectAtoms(res_name='LYS')
    sele_ca_asp=sele_ca & sele_asp
    sele_ca_glu=sele_ca & sele_glu
    sele_ca_hsp=sele_ca & ( sele_hsp | sele_hsd | sele_hse )
    sele_ca_lys=sele_ca & sele_lys
    sele_asp_resid=sele_ca_asp.get_res_ids()
    sele_glu_resid=sele_ca_glu.get_res_ids()
    sele_hsp_resid=sele_ca_hsp.get_res_ids()
    sele_lys_resid=sele_ca_lys.get_res_ids()
    sele_asp_resinx=sele_ca_asp.get_res_indexes()
    sele_glu_resinx=sele_ca_glu.get_res_indexes()
    sele_hsp_resinx=sele_ca_hsp.get_res_indexes()
    sele_lys_resinx=sele_ca_lys.get_res_indexes()
    titr_res_dict={'ASP':[sele_asp_resid,sele_asp_resinx],
                   'GLU':[sele_glu_resid,sele_glu_resinx],
                   'HSP':[sele_hsp_resid,sele_hsp_resinx],
                   'LYS':[sele_lys_resid,sele_lys_resinx]}
    print(titr_res_dict)
    return titr_res_dict


In [14]:
def titr_grp(resn):
    '''
    atom names in the titratable group for a given amino acid
    '''
    if resn == 'ASP' or resn == 'ASPM': 
       type_list=['CB','HB1','HB2','CG','OD1','OD2','HD1','HD2']
    elif resn == 'GLU' or resn == 'GLUM':
       type_list=['CG','HG1','HG2','CD','OE1','OE2','HE1','HE2']
    elif resn == 'HSP' or resn == 'HSD' or resn == 'HSE':
       type_list=['CB','HB1','HB2','CG','ND1','HD1','CE1','HE1','CD2','HD2','NE2','HE2']
    elif resn == 'LYS':
       type_list=['CE','HE1','HE2','NZ','HZ1','HZ2','HZ3']
    return type_list

In [15]:
def add_suffix(resn,suffix):
    '''
    add suffix to titratable groups, and write the pdb file
    '''
    atom_list=titr_grp(resn.upper())
    for name in atom_list:
        lingo.charmm_script('rename atom {}{} sele type {} .and. resn {} end'.format(name,suffix,name,resn))

In [16]:
class state0:
    '''
    generate state0 structure: default protonation & add atom name suffix.
    '''
    def __init__(self,titr_res):
        self.titr_res_dict=titr_res
        if 'HSP' in list(self.titr_res_dict.keys()):
           lingo.charmm_script('rename resn HSP sele resn HSD .or. resn HSE end')
           write.coor_pdb('tmp.pdb')
           psf.delete_atoms()
           read.sequence_pdb('tmp.pdb')
           gen.new_segment(seg_name=segid,first_patch='ACE',last_patch='CT3',setup_ic=True)
           read.pdb('tmp.pdb',resid=True)
           ic.prm_fill(replace_all=False)
           ic.build()
           os.system('rm -f tmp.pdb')
        add_suffix('ASP', 'w')
        add_suffix('GLU', 'w')
        add_suffix('HSP', 'w')
        add_suffix('LYS', 'w')
        write.coor_pdb(out_dir+'/'+fname_base+'-state0.pdb')
        write.psf_card(out_dir+'/'+fname_base+'-state0.psf')

In [17]:
def solvate_prot(nchrg,salt_conc=0):
    
    # solvate the structure
    solv='convpdb.pl -solvate -cubic -cutoff 12 -out charmm22 {}/{}.pdb | convpdb.pl -segnames > {}/prot-solv.pdb'.format(out_dir,fname_base,out_dir)
    o=bytes.decode(subprocess.check_output(solv,shell=True,stderr=subprocess.STDOUT)).split()
    size=float(o[len(o)-1])
    np.savetxt('{}/box.dat'.format(out_dir),np.array([size]))

    # number of water molecules
    count_wat='grep "OH2 TIP3" {}/prot-solv.pdb | wc -l'.format(out_dir)
    o=bytes.decode(subprocess.check_output(count_wat,shell=True,stderr=subprocess.STDOUT)).split()
    nwat=int(o[0])
    
    # number of ions
    nion=round(salt_conc/55.5*nwat)
    nSOD=nion
    nCLA=nion
    if nchrg > 0: 
       nCLA+=int(np.abs(np.floor(nchrg)))
    else:
       nSOD+=int(np.abs(np.ceil(nchrg)))
    print("number of charges in protein: {}, ions: {}, NA: {}, CL: {}".format(nchrg,nion,nSOD,nCLA))
    
    # solvate again with ions
    solv2='convpdb.pl -solvate -cubic -cutoff 12 -out charmm22 -ions CLA:{}=SOD:{} {}/{}.pdb | convpdb.pl -segnames > {}/prot-solv2.pdb'.format(nCLA,nSOD,out_dir,fname_base,out_dir)
    o=bytes.decode(subprocess.check_output(solv2,shell=True,stderr=subprocess.STDOUT))
    
    # separate out water and ions to read into new segments
    getwat = 'convpdb.pl -out charmm22 -nsel TIP3 -segnames -renumwatersegs {}/prot-solv2.pdb > {}/water.pdb'.format(out_dir,out_dir)
    os.system(getwat)
    if nCLA+nSOD > 0:
        getions = 'convpdb.pl -out charmm22 -nsel CLA+SOD -segnames {}/prot-solv2.pdb | sed "s/HETATM/ATOM  /g" | sed "s/ HETA/ IONS/g" > {}/ions.pdb'.format(out_dir,out_dir)
        os.system(getions)
   
    # get water segid(s)
    water_pdb=open('{}/water.pdb'.format(out_dir),'r')
    wat_segid=[]
    segid_i=''
    for line in water_pdb:
        if line.split()[0]=='ATOM':
           segid_j=line.split()[-1]
           if segid_j != segid_i:
               wat_segid.append(segid_j)
           segid_i=segid_j
    print('water segids are ',wat_segid)
    
    # read water into charmm
    for segid_w in wat_segid:
        read.sequence_pdb('{}/water.pdb'.format(out_dir),segid=segid_w)
        # note that the option should be "angle" and "dihedral", not "angles" or "dihedrals" !!!
        gen.new_segment(seg_name=segid_w,angle=False,dihedral=False)
    read.pdb('{}/water.pdb'.format(out_dir),resid=True)

    # read ions into charmm
    if nCLA+nSOD > 0:
        read.sequence_pdb('{}/ions.pdb'.format(out_dir),segid='IONS')
        # note that the option should be "angle" and "dihedral", not "angles" or "dihedrals" !!!
        gen.new_segment(seg_name='IONS',angle=False,dihedral=False)
        read.pdb('{}/ions.pdb'.format(out_dir),resid=True)

    # remove water molecules that overlap with protein/ligand/ions/cyrstal-water
    for segid_w in wat_segid:
        lingo.charmm_script('delete atom sele .byres. ( segid {} .and. ( ( segid {}* .or. segid ION* .or. segid SOLV ) .around. 2.8 )) end'.format(segid_w,segid))

    # write pdb and psf files
    write.psf_card('{}/solv-final.psf'.format(out_dir))
    write.coor_pdb('{}/solv-final.pdb'.format(out_dir))

    # check if there is any missing coordinate
    debug=True
    if debug == True:
       sele_nan=~pycharmm.SelectAtoms(initials=True)
       sele_nan_n=sele_nan.get_n_selected()
       if sele_nan_n > 0:
          print(f"ERROR! coordinates of {sele_nan_n} atoms were not initialized")
          quit()

In [18]:
def coor_shift(titr_res):
    for resn in list(titr_res.keys()):
        sele_m=~pycharmm.SelectAtoms(select_all=True)
        sele_p=~pycharmm.SelectAtoms(select_all=True)
        sele_u=~pycharmm.SelectAtoms(select_all=True)
        atom_list=titr_grp(resn)
        for atom_name in atom_list:
            sele_m = sele_m | pycharmm.SelectAtoms().by_atom_type(atom_name+'M')
            sele_p = sele_p | pycharmm.SelectAtoms().by_atom_type(atom_name+'P')
            sele_u = sele_u | pycharmm.SelectAtoms().by_atom_type(atom_name+'U')
            sele_m = sele_m & pycharmm.SelectAtoms().by_res_name(resn)
            sele_p = sele_p & pycharmm.SelectAtoms().by_res_name(resn)
            sele_u = sele_u & pycharmm.SelectAtoms().by_res_name(resn)
        sele_nameM='site{}subM'.format(resn)
        sele_nameP='site{}subP'.format(resn)
        sele_nameU='site{}subU'.format(resn)
        sele_m.store(sele_nameM)
        sele_p.store(sele_nameP)
        sele_u.store(sele_nameU)
        lingo.charmm_script('coor print sele %s end'%(sele_nameM))
        lingo.charmm_script('coor print sele %s end'%(sele_nameP))
        lingo.charmm_script('coor print sele %s end'%(sele_nameU))
        nm=sele_m.get_n_selected()
        np=sele_p.get_n_selected()
        nu=sele_u.get_n_selected()
        if nm > 0:
           lingo.charmm_script('''
                 coor tranlate xdir 1 ydir 0 zdir 0 dist 0.1 sele {} end
                 '''.format(sele_nameM))
        if np > 0:
           lingo.charmm_script('''
                 coor tranlate xdir 0 ydir 1 zdir 0 dist 0.1 sele {} end
                 '''.format(sele_nameP))
        if nu > 0:
           lingo.charmm_script('''
                 coor tranlate xdir 0 ydir 0 zdir 1 dist 0.1 sele {} end
                 '''.format(sele_nameU))

Personally, I prefer saving CHARMM output to a file

In [19]:
open_clog()

 VOPEN> Attempting to open::./test2/charmm.out::
  
 CHARMM>     outu 20


In [20]:
read_param()

Here we are going  to generate psf for a given sequence, and generate initial structure based on internal coordinates.
This function will also return the total number of charges of the system

In [21]:
nchrg=gen_init()

For this tutorial, the salt concentration is 0

In [22]:
solvate_prot(nchrg,0)

number of charges in protein: 1, ions: 0, NA: 0, CL: 1


parseSelection: cannot find residue or atom SOD in current structure


water segids are  ['WT00']


Delete the whole system


In [23]:
psf.delete_atoms(selection=pycharmm.SelectAtoms(select_all=True))

True

In [24]:
nchrg=gen_init()

Find all titratable residues in a protein


In [25]:
titr_res=find_titr_res()

{'ASP': [[], []], 'GLU': [['4'], [3]], 'HSP': [['5'], [4]], 'LYS': [['1'], [0]]}


For Glu and Asp, add H to one O atom of the carboxylate group


In [26]:
state1_o2(titr_res)

For Glu and Asp, add H to the other O atom of the carboxylate group


In [27]:
state2_o1()


In [28]:
psf.delete_atoms(selection=pycharmm.SelectAtoms(select_all=True))

True

In [29]:
nchrg=gen_init()

Standard protonation state, and add atom name suffix.

In [30]:
state0(titr_res)

Build a hybrid system

In [31]:
patch_all(titr_res,3)

A selection has been stored as SITEASPSUBM
A selection has been stored as SITEASPSUBP
A selection has been stored as SITEASPSUBU
A selection has been stored as SITEGLUSUBM
A selection has been stored as SITEGLUSUBP
A selection has been stored as SITEGLUSUBU
A selection has been stored as SITEHSPSUBM
A selection has been stored as SITEHSPSUBP
A selection has been stored as SITEHSPSUBU
A selection has been stored as SITELYSSUBM
A selection has been stored as SITELYSSUBP
A selection has been stored as SITELYSSUBU


Solvate the hybrid system

In [32]:
def solvate():
    inp_pdb_fn=f'{out_dir}/solv-final.pdb'
    for seg_id in ['WT00']:
        read.sequence_pdb(inp_pdb_fn,segid=seg_id)
        # note that the valid option is "angle" not "angles"
        gen.new_segment(seg_name=seg_id,angle=False,dihedral=False)
    #settings.set_bomb_level(-1)
    read.pdb(inp_pdb_fn,resid=True)
    write.psf_card(f'{out_dir}/{out_fn}-solv.psf')
    write.coor_pdb(f'{out_dir}/{out_fn}-solv.pdb')
    settings.set_bomb_level(0)
    energy.show()

In [33]:
solvate()

Remove temporary files

In [34]:
os.system(f'rm {out_dir}/{fname_base}-state*-o*.p*')
os.system(f'rm {out_dir}/{fname_base}-state0.p*')
os.system(f'rm {out_dir}/{fname_base}-3state.p*')
os.system(f'rm {out_dir}/solv-final.*')
os.system(f'rm {out_dir}/{fname_base}.p*')
os.system(f'rm {out_dir}/prot-solv*.pdb')
os.system(f'rm {out_dir}/water.pdb')
os.system(f'rm {out_dir}/ions.pdb')



0